In [3]:
from sfm.data.sci_data.SFMDecTokenizer import SFMDecTokenizer
import torch
import os

In [4]:
tokenizer_home = '/hai1/ds_dataset/llama2/llama-2-7b'
tokenizer = SFMDecTokenizer.from_pretrained(
    tokenizer_home,
    prot_spm_path='/blob/shufxi/data/scigpt/ur50bpe/bpe',
    dna_spm_path='/blob/shufxi/data/scigpt/dnabpe/bpe',
    rna_spm_path='/blob/shufxi/data/scigpt/rnabpe/bpe',
)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LlamaTokenizer'. 
The class this function is called from is 'SFMDecTokenizer'.


[2024-03-13 03:01:56.880][INFO]: Loading protein sentencepiece model from /blob/shufxi/data/scigpt/ur50bpe/bpe.model and /blob/shufxi/data/scigpt/ur50bpe/bpe.vocab
[2024-03-13 03:01:57.848][INFO]: Loading DNA sentencepiece model from /blob/shufxi/data/scigpt/dnabpe/bpe.model and /blob/shufxi/data/scigpt/dnabpe/bpe.vocab
[2024-03-13 03:01:58.108][INFO]: Loading RNA sentencepiece model from /blob/shufxi/data/scigpt/rnabpe/bpe.model and /blob/shufxi/data/scigpt/rnabpe/bpe.vocab
[2024-03-13 03:01:58.617][INFO]: Tokenizer has 40014 tokens


In [3]:
tokenizer.tokenize('<rna>GCCGGCGUAGCUCAGUUGGUAGAGCAAUUGUUUUGUAAACAAAAGGUCGGGGGUUCGAUUCCUCUCGCCGGCU</rna>')

['<rna>',
 '<r>GCCG',
 '<r>GCGU',
 '<r>AGCUC',
 '<r>AGUUGG',
 '<r>UAG',
 '<r>AGCAAU',
 '<r>UGUU',
 '<r>UUGUAAA',
 '<r>CAAA',
 '<r>AGGU',
 '<r>CGGG',
 '<r>GGUUCGAUUCC',
 '<r>UCUC',
 '<r>GCC',
 '<r>GGCU',
 '</rna>']

In [4]:
s = tokenizer.tokenize('The <protein>EMMFEQTFKNID</protein> and the compound <mol>CCC</mol> can interact with each other.')
print(' '.join(s))

▁The <protein> <a>E <a>M <a>M <a>FEQ <a>TFK <a>NID </protein> ▁and ▁the ▁comp ound <mol> <m>C <m>C <m>C </mol> ▁can ▁interact ▁with ▁each ▁other .


In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM
# ckpt_home = '/hai1/shufxi/scigpt/7bv2/stageB/global_step26999/'

# ckpt_home = '/hai1/shufxi/scigpt/7bv3/stageB/global_step11999'
# ckpt_home = '/blob/shufxi/scigpt/7bv3/inst/20240227121523/global_step3585/'
# ckpt_home = '/blob/shufxi/scigpt/7bv3/prot/20240228025826/global_step11715'
# ckpt_home = '/hai1/shufxi/scigpt/7bv3/stageB.prot/global_step105720'
ckpt_home = '/blob/shufxi/scigpt/7bv3/inst/20240311061510/global_step200'

def show_ckpt(name, ckpt):
    for k, v in ckpt.items():
        if 'dummy' not in k:
            print(name, k, v.shape)

model = AutoModelForCausalLM.from_pretrained(tokenizer_home)

model_dict = model.state_dict()
ckpt_dict = {}
layer0 = torch.load(os.path.join(ckpt_home, "layer_00-model_states.pt"), map_location=torch.device("cpu"))
ckpt_dict['model.embed_tokens.weight'] = layer0['embed_tokens.weight']
show_ckpt('layer0', layer0)

for l in range(0, 32):
    l_index = str(l + 1).zfill(2)
    layer = torch.load(os.path.join(ckpt_home, f"layer_{l_index}-model_states.pt"), map_location=torch.device("cpu"))
    show_ckpt(l_index, layer)
    for k in layer:
        if "dummy" in k or 'rotary_emb' in k:
            continue
        ckpt_dict[f"model.layers.{l}.{k}"] = layer[k]
layer = torch.load(os.path.join(ckpt_home, "layer_33-model_states.pt"), map_location=torch.device("cpu"))
show_ckpt(33, layer)
ckpt_dict["model.norm.weight"] = layer["norm.weight"]

layer = torch.load(os.path.join(ckpt_home, "layer_34-model_states.pt"), map_location=torch.device("cpu"))
show_ckpt(33, layer)
ckpt_dict["lm_head.weight"] = layer["lm_head.weight"]
model_dict.update(ckpt_dict)

model.resize_token_embeddings(len(tokenizer))
model.load_state_dict(model_dict)

Loading checkpoint shards: 100%|██████████| 2/2 [00:26<00:00, 13.48s/it]


layer0 embed_tokens.weight torch.Size([40014, 4096])
01 self_attn.q_proj.weight torch.Size([4096, 4096])
01 self_attn.k_proj.weight torch.Size([4096, 4096])
01 self_attn.v_proj.weight torch.Size([4096, 4096])
01 self_attn.o_proj.weight torch.Size([4096, 4096])
01 mlp.gate_proj.weight torch.Size([11008, 4096])
01 mlp.up_proj.weight torch.Size([11008, 4096])
01 mlp.down_proj.weight torch.Size([4096, 11008])
01 input_layernorm.weight torch.Size([4096])
01 post_attention_layernorm.weight torch.Size([4096])
02 self_attn.q_proj.weight torch.Size([4096, 4096])
02 self_attn.k_proj.weight torch.Size([4096, 4096])
02 self_attn.v_proj.weight torch.Size([4096, 4096])
02 self_attn.o_proj.weight torch.Size([4096, 4096])
02 mlp.gate_proj.weight torch.Size([11008, 4096])
02 mlp.up_proj.weight torch.Size([11008, 4096])
02 mlp.down_proj.weight torch.Size([4096, 11008])
02 input_layernorm.weight torch.Size([4096])
02 post_attention_layernorm.weight torch.Size([4096])
03 self_attn.q_proj.weight torch.Size

<All keys matched successfully>

In [6]:
model = model.cuda()

In [10]:
import glob
from tqdm import tqdm

In [30]:
for path in glob.glob('/blob/shufxi/data/scigpt/inst_text/test_*'):
    input_data = []
    with open(path, 'r') as f:
        for line in f:
            input_data.append(line.strip())
    
    output_path = path.replace('test', 'output')
    with open(output_path, 'w') as f:
        for line in tqdm(input_data, desc=path):
            question = line.split('\t')[0]
            prompt = f'Instruction: {question}\n\n\nResponse:'
            input_ids = tokenizer(prompt, return_tensors='pt').input_ids.cuda()
            output = model.generate(
                input_ids,
                num_beams=4,
                max_new_tokens=100,
                num_return_sequences=4,
                return_dict_in_generate=True,
                output_scores=True,
                do_sample=False,
            )
            output_ids = output.sequences[0]
            output_text = tokenizer.decode(output_ids)
            output_begin_idx = output_text.rfind('Response:') + len('Response:')
            response_end_idx = output_text.find('</s>')
            output_text = output_text[output_begin_idx:response_end_idx].strip()
            f.write(output_text + '\n')

/blob/shufxi/data/scigpt/inst_text/test_chemical_disease_interaction_extraction.tsv:   0%|          | 0/50 [00:00<?, ?it/s]

/blob/shufxi/data/scigpt/inst_text/test_chemical_disease_interaction_extraction.tsv: 100%|██████████| 50/50 [01:46<00:00,  2.12s/it]
/blob/shufxi/data/scigpt/inst_text/test_chemical_entity_recognition.tsv: 100%|██████████| 100/100 [01:26<00:00,  1.15it/s]
/blob/shufxi/data/scigpt/inst_text/test_chemical_protein_interaction_extraction.tsv: 100%|██████████| 78/78 [06:53<00:00,  5.30s/it]
/blob/shufxi/data/scigpt/inst_text/test_multi_choice_question.tsv: 100%|██████████| 1001/1001 [05:59<00:00,  2.78it/s]
/blob/shufxi/data/scigpt/inst_text/test_open_question.tsv: 100%|██████████| 1004/1004 [07:02<00:00,  2.38it/s]
/blob/shufxi/data/scigpt/inst_text/test_true_or_false_question.tsv: 100%|██████████| 100/100 [02:18<00:00,  1.38s/it]


: 

In [97]:
tokenizer.tokenize('<protein>MSSANFTDGPWSFEHDDSTSSIIQVFSLSDSAKGDPDICRLQHEPDACLISTAPQMYSLLKDAMVKLSFGDYESKLISDDIKSLLAKARGEHV')

['<protein>',
 '<a>M',
 '<a>SSA',
 '<a>NFT',
 '<a>DG',
 '<a>PW',
 '<a>SF',
 '<a>EH',
 '<a>DD',
 '<a>STS',
 '<a>SII',
 '<a>QV',
 '<a>FSL',
 '<a>SDS',
 '<a>AKG',
 '<a>DPD',
 '<a>IC',
 '<a>RLQ',
 '<a>HE',
 '<a>PDA',
 '<a>CL',
 '<a>IST',
 '<a>APQ',
 '<a>MY',
 '<a>SLL',
 '<a>KDA',
 '<a>MV',
 '<a>KL',
 '<a>SFG',
 '<a>DYE',
 '<a>SKL',
 '<a>ISD',
 '<a>DIK',
 '<a>SLL',
 '<a>AKA',
 '<a>RGE',
 '<a>HV']

In [101]:
input_ids = tokenizer('<protein>MSSANFTDGPWSFEHDDSTSSIIQVFSLSDSAKGDPDICRLQHEPDACLISTAPQMYSLLKDAMVKLSFGDYESKLISDDIK', return_tensors="pt").input_ids.cuda()
output = model.generate(
    input_ids,
    num_beams=4,
    max_new_tokens=100,
    num_return_sequences=4,
    return_dict_in_generate=True,
    output_scores=True,
    do_sample=True,
    top_p=0.95,
    # no_repeat_ngram_size=2,
    temperature=1.2,
    # penalty_alpha = 0.9
)

for i in range(output.sequences.shape[0]):
    s = tokenizer.decode(output.sequences[i])
    s = s.replace(' <a>', '').replace(' <m>', '')
    print(s)
    print("=====================================")

<s> <protein>MSSANFTDGPWSFEHDDSTSSIIQVFSLSDSAKGDPDICRLQHEPDACLISTAPQMYSLLKDAMVKLSFGDYESKLISDDIKAAMAKAGK </protein> </s><unk><unk>
<s> <protein>MSSANFTDGPWSFEHDDSTSSIIQVFSLSDSAKGDPDICRLQHEPDACLISTAPQMYSLLKDAMVKLSFGDYESKLISDDIKAAMAKATQ </protein> </s><unk><unk>
<s> <protein>MSSANFTDGPWSFEHDDSTSSIIQVFSLSDSAKGDPDICRLQHEPDACLISTAPQMYSLLKDAMVKLSFGDYESKLISDDIKAAMDAMEAAE </protein> </s><unk>
<s> <protein>MSSANFTDGPWSFEHDDSTSSIIQVFSLSDSAKGDPDICRLQHEPDACLISTAPQMYSLLKDAMVKLSFGDYESKLISDDIKAAMAKAGKMEA </protein> </s>


In [106]:
def compute_entropy(logits):
    logits = logits.squeeze(0) # assume batch size 1
    probs = torch.nn.functional.softmax(logits, dim=-1)
    log_probs = torch.nn.functional.log_softmax(logits, dim=-1)
    return -torch.sum(probs * log_probs, dim=-1)

def binary_search_t(logits, target_entropy, tol=1e-3, max_iter=100):
    low = 0
    high = 1e3

    for i in range(max_iter):
        t = (low + high) / 2
        entropy = compute_entropy(logits / t).item()
        if abs(entropy - target_entropy) < tol:
            return t
        elif entropy > target_entropy:
            high = t
        else:
            low = t
    return t

def stable_decoding(input_text, max_new_tokens):
    inputs = tokenizer(input_text, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    attention_mask = inputs["attention_mask"].cuda()

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        prompt_entropy = compute_entropy(logits[:, :-1, :]).mean().item()

    t_list = []
    p_list = []


    prompt_entropy = 5
    for i in range(max_new_tokens):
        t = binary_search_t(logits[:, -1, :], prompt_entropy)
        t_list.append(t)
        probs = torch.nn.functional.softmax(logits[:, -1, :] / t, dim=-1)
        sampled_token = torch.multinomial(probs[0], 1)
        p_list.append(probs[0, sampled_token].item())
        # check eos token
        if sampled_token == tokenizer.eos_token_id:
            break
        input_ids = torch.cat([input_ids, sampled_token.unsqueeze(0)], dim=-1)
        attention_mask = torch.cat([attention_mask, torch.ones(1, 1).cuda()], dim=-1)
        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits

    return {
        'prompt_entropy': prompt_entropy,
        't_list': t_list,
        'p_list': p_list,
        'output_ids': input_ids[0],
        'output_tokens': [tokenizer.decode(input_ids[0, i]) for i in range(input_ids.shape[1])],
        'output_text': tokenizer.decode(input_ids[0], skip_special_tokens=True).replace(' <a>', '').replace(' <m>', '')
    }


s = stable_decoding('<protein>MSSANFTDGPWSFEHDDSTSSIIQV', 50)


In [107]:
for k, v in s.items():
    print(k, '\n', v)

prompt_entropy 
 5
t_list 
 [0.6350278854370117, 0.46539306640625, 0.521540641784668, 0.7226467132568359, 0.6462335586547852, 0.4540681838989258, 0.5038976669311523, 0.5255937576293945, 0.5115270614624023, 0.7340908050537109, 0.644683837890625, 0.4894733428955078, 0.48732757568359375, 0.5574226379394531, 0.4271268844604492, 0.6384849548339844, 0.4526376724243164, 0.44274330139160156, 0.41103363037109375, 0.4734992980957031, 0.47206878662109375, 0.504612922668457, 0.4878044128417969, 0.5767345428466797, 0.535130500793457, 0.5605220794677734, 0.461578369140625, 0.5095005035400391, 0.45561790466308594, 0.7429122924804688, 0.9198188781738281, 1.0519027709960938, 0.5536079406738281, 0.6978511810302734, 0.6344318389892578, 0.6392002105712891, 0.744938850402832, 0.6874799728393555, 0.548243522644043, 0.6982088088989258, 0.5599260330200195, 0.5686283111572266, 0.7567405700683594, 0.6015300750732422, 0.6575584411621094, 0.7243156433105469, 0.5462169647216797, 0.5232095718383789, 0.5435943603515

In [82]:
def my_decoding(input_text, max_new_tokens, eps=1e-6):
    inputs = tokenizer(input_text, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    attention_mask = inputs["attention_mask"].cuda()

    for i in range(max_new_tokens):
        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
        probs = torch.nn.functional.softmax(logits[:, -1, :], dim=-1)
        max_prob = probs.max().item()
        
        penalty = max_prob / (probs + eps)
        
        new_prob = probs / penalty
        
        sampled_token = torch.multinomial(new_prob[0], 1)
        # check eos token
        if sampled_token == tokenizer.eos_token_id:
            break
        input_ids = torch.cat([input_ids, sampled_token.unsqueeze(0)], dim=-1)
        attention_mask = torch.cat([attention_mask, torch.ones(1, 1).cuda()], dim=-1)


    return {
        'output_ids': input_ids[0],
        'output_tokens': [tokenizer.decode(input_ids[0, i]) for i in range(input_ids.shape[1])],
        'output_text': tokenizer.decode(input_ids[0], skip_special_tokens=True).replace(' <a>', '').replace(' <m>', '')
    }
    
s = my_decoding('<protein>', 100)
for k, v in s.items():
    print(k, '\n', v)

output_ids 
 tensor([    1, 32007, 33873, 33925, 35740, 33873, 33873, 33873, 33873, 33873,
        33873, 33873, 33873, 33873, 33873, 33873, 33873, 33873, 33873, 33873,
        33873, 33873, 33873, 33873, 33873, 33873, 33873, 33873, 33873, 33873,
        33873, 33873, 33873, 33873, 33873, 33873, 33873, 33873, 33873, 33873,
        33873, 33873, 33873, 33873, 33873, 33873, 33873, 33873, 33873, 33873,
        33873, 33873, 33873, 33873, 33873, 33873, 33873, 33873,   278,   349,
        29902, 29899, 29941, 29907, 29899, 29946, 29889, 29929, 29941, 29899,
        29941, 29889, 29896, 29899, 29946, 29947, 29941, 29889,   450,  3677,
          747, 29875, 13574, 26823,   317, 29896, 29900, 29900, 29953, 29889,
        29896, 29889, 29900, 29953, 29899, 29941, 29929, 29889, 29896, 29889,
        29900, 29871], device='cuda:0')
output_tokens 
 ['<s>', '<protein>', '<a>M', '<a>GA', '<a>FIL', '<a>M', '<a>M', '<a>M', '<a>M', '<a>M', '<a>M', '<a>M', '<a>M', '<a>M', '<a>M', '<a>M', '<a>M', '<a>M',

In [66]:
input_ids = tokenizer('Instruction: Analyze and describe the functional role of the corresponding protein for the sequence <protein>MTEFESAPAYQEAKYLTSAAEFDQLPPDQGAEIAFIGRSNAGKSSALNIITGIKGLARTSKTPGRTQMINFFALNEHERLVDLPGYGYAKVPRMVQKRWEELVDSYLKKRRCLKGLVVVMDIRHPLKEMDEDVIEWAVNYDIPIHILLTKSDKLSQNAAKKTLGEVQTAISAYGEKLTLQLFSSHDRTGLDEVKAVLSQWFRSEP</protein>.\n\n\nResponse:', return_tensors="pt").input_ids.cuda()
output = model.generate(
    input_ids,
    num_beams=4,
    max_new_tokens=300,
    num_return_sequences=4,
    return_dict_in_generate=True,
    output_scores=True,
    do_sample=True,
    top_p=0.95,
)

for i in range(output.sequences.shape[0]):
    s = tokenizer.decode(output.sequences[i])
    s = s.replace(' <a>', '').replace(' <m>', '')
    print(s)
    print("=====================================")

<s>Instruction: Analyze and describe the functional role of the corresponding protein for the sequence <protein>MTEFESAPAYQEAKYLTSAAEFDQLPPDQGAEIAFIGRSNAGKSSALNIITGIKGLARTSKTPGRTQMINFFALNEHERLVDLPGYGYAKVPRMVQKRWEELVDSYLKKRRCLKGLVVVMDIRHPLKEMDEDVIEWAVNYDIPIHILLTKSDKLSQNAAKKTLGEVQTAISAYGEKLTLQLFSSHDRTGLDEVKAVLSQWFRSEP </protein>  .


Response: The sequence of this</s><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><u

In [ ]:
import random

In [ ]:
def chat(instruction: str, beam:int=4, do_sample:bool=True, top_p:float=0.9):
    prompt = f"Instruction: {instruction}\n\n\nResponse:"
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.cuda()
    output = model.generate(
        input_ids,
        num_beams=beam,
        max_new_tokens=50,
        num_return_sequences=1,
        return_dict_in_generate=True,
        output_scores=True,
        do_sample=do_sample,
        top_p=top_p,
        repetition_penalty=2.0
    )
    ret = tokenizer.decode(output.sequences[0])
    start_idx = ret.find('Response:') + len('Response:')
    end_idx = ret.find('</s>')

    ret = ret[start_idx:end_idx]
    ret = ret.replace(' <m>', '').replace(' <a>', '')

    return ret

In [ ]:
chat('design a transmembrane protein')

In [ ]:
chat('design a transmembrane protein')

In [ ]:
chat('design a protein that fold fast')

In [ ]:
def compute_ppl(seq):
    input_ids = tokenizer(seq, return_tensors="pt").input_ids.cuda()
    labels = input_ids.clone()

    with torch.no_grad():
        outputs = model(input_ids, labels=labels, return_dict=True) # shift inside
    loss = outputs.loss
    return loss.item(), torch.exp(loss).item()

compute_ppl('<mol>CCN(CC)CCCC(C)Nc1ccnc2cc(Cl)ccc12</mol>')

In [ ]:
print(compute_ppl('<protein>MEFLGLW'))
print(compute_ppl('<protein>MEFLGM'))
print(compute_ppl('<protein>MTTCRISQTARFNLPGIIAFESAKRGGELLDVPDLGNPPEDWEMLNPDGEYGTNPVVDEIVKKS</protein>'))

In [ ]:
torch.softmax(output.scores[0][0], dim=0).sort(descending=True)

In [ ]:
output.scores[0].shape

In [ ]:
output.sequences[0]

In [ ]:
input_ids = tokenizer('An RNA like <rna>', return_tensors="pt").input_ids.cuda()
n = 4
output = model.generate(
    input_ids,
    num_beams=n,
    do_sample=True,
    max_new_tokens=100,
    num_return_sequences=n,
    return_dict_in_generate=True,
    output_scores=True,
    top_k=10
)

for i in range(n):
    s = tokenizer.decode(output.sequences[i])
    # print(s)
    # mol = s[s.find('<rna>') + len('<rna>'):s.find('</rna>')]
    s = s.replace('<r>', '')
    print(s)

In [ ]:
input_ids = tokenizer('<rna>GCCGGCGUAGCUCAGUUGG', return_tensors="pt").input_ids.cuda()
n = 10
output = model.generate(
    input_ids,
    num_beams=n,
    do_sample=False,
    # top_p=0.95,
    max_new_tokens=100,
    num_return_sequences=n,
    return_dict_in_generate=True,
    output_scores=True
)

for i in range(n):
    s = tokenizer.decode(output.sequences[i])
    s = s.replace('<r>', '').replace(' ', '')
    print(s)

In [ ]:
input_ids = tokenizer('Elements, energy above hull and band gap to material: The material contains Ba, B, Pb, O. Its energy above hull is 0.01 eV/atom. Its band gap is 4.1 eV. The material is <material>', return_tensors="pt").input_ids.cuda()
n = 4
output = model.generate(
    input_ids,
    num_beams=n,
    do_sample=False,
    max_new_tokens=200,
    num_return_sequences=n,
    return_dict_in_generate=True,
    output_scores=True,
)

for i in range(n):
    s = tokenizer.decode(output.sequences[i])
    # print(s)
    # mol = s[s.find('<rna>') + len('<rna>'):s.find('</rna>')]
    s = s.replace('<i>', '')
    print(s)

In [ ]:
chat('design a protein that folds fast')